# Домашняя работа 1

### Задание 1. Создание структуры базы данных

In [12]:
# импорт библиотек
import pandas as pd
import numpy as np

![image](img/img_1.png)

### Задание 2. Исследование и нормализация данных

In [3]:
# чтение данных по транзакциям

transactions = pd.read_csv('customer_and_transaction/transaction-Tаблица 1.csv', sep=';')
transactions.head(2)

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost
0,1,2,2950,2/25/17,False,Approved,Solex,Standard,medium,medium,"71,49","53,62"
1,2,3,3120,5/21/17,True,Approved,Trek Bicycles,Standard,medium,large,"2091,47","388,92"


In [4]:
# чтение данных по пользователям

customers = pd.read_csv('customer_and_transaction/customer-Tаблица 1.csv', sep=';')
customers.head(2)

,customer_id,first_name,last_name,gender,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,address,postcode,state,country,property_valuation
0,1,Laraine,Medendorp,F,1953-10-12,Executive Secretary,Health,Mass Customer,N,Yes,060 Morning Avenue,2016,New South Wales,Australia,10
1,2,Eli,Bockman,Male,1980-12-16,Administrative Officer,Financial Services,Mass Customer,N,Yes,6 Meadow Vale Court,2153,New South Wales,Australia,10


In [6]:
# создание таблицы по товарам

goods = transactions[[
    'product_id',
    'brand',
    'product_line',
    'product_class',
    'product_size',
    'list_price',
    'standard_cost'
]]

Исследуем данные в таблице **goods**. Данные ненормализованы.

In [7]:
# поиск дубликатов строк

n_rows = goods.shape[0]
n_rows_wo_duplicates = goods.drop_duplicates().shape[0]

print(f'Количество строк в исходной таблице - {n_rows}')
print(f'Количество строк после удаления всех дубликатов - {n_rows_wo_duplicates}')


Количество строк в исходной таблице - 20000
Количество строк после удаления всех дубликатов - 397


Данные не находятся в 1НФ из-за наличия дубикатов

In [8]:
# удаление дубликатов

goods = goods.drop_duplicates()

Для сведения данных к 2НФ найдем ключевые поля. Для этого найдем уникальные элементы, определенные единственным образом, которые можем использовать в качестве первичного ключа.

In [9]:
n_rows = goods.shape[0]
n_products = goods.product_id.nunique()
n_brands = goods.brand.nunique()
n_product_line = goods.product_line.nunique()
n_product_class = goods.product_class.nunique()
n_product_size = goods.product_size.nunique()
n_list_price = goods.list_price.nunique()
n_standard_cost = goods.standard_cost.nunique()

print(f'Количество уникальных строк - {n_rows}')
print(f'Количество уникальных продуктов - {n_products}')
print(f'Количество брендов - {n_brands}')
print(f'Количество продуктовых линеек - {n_product_line}')
print(f'Количество классов продуктов - {n_product_class}')
print(f'Количество размеров продуктов - {n_product_size}')
print(f'Количество уникальных значений list price - {n_list_price}')
print(f'Количество уникальных значений standard cost - {n_standard_cost}')

Количество уникальных строк - 397
Количество уникальных продуктов - 101
Количество брендов - 6
Количество продуктовых линеек - 4
Количество классов продуктов - 3
Количество размеров продуктов - 3
Количество уникальных значений list price - 296
Количество уникальных значений standard cost - 100


In [10]:
# проверка на наличие пустых строк

goods.isna().sum()

product_id         0
brand            196
product_line     196
product_class    196
product_size     196
list_price         0
standard_cost    196
dtype: int64

Ни одно из указанных полей не может быть первичным ключом из-за несоответствия количества строк количеству уникальных элементов полей, а также из-за наличия пропусков в данных. Для приведения данных к 2НФ создадим поле **goods_id** целого типа, которое будет уникальным первичным ключом.

In [13]:
goods['goods_id'] = np.arange(1, len(goods)+1)
goods.head()

,product_id,brand,product_line,product_class,product_size,list_price,standard_cost,goods_id
0,2,Solex,Standard,medium,medium,"71,49","53,62",1
1,3,Trek Bicycles,Standard,medium,large,"2091,47","388,92",2
2,37,OHM Cycles,Standard,low,medium,"1793,43","248,82",3
3,88,Norco Bicycles,Standard,medium,medium,"1198,46","381,10",4
4,78,Giant Bicycles,Standard,medium,large,"1765,3","709,48",5


Ввиду малочисленности уникальных данных в столбцах **brand**, **product_line**, **product_class**, **product_size** нецелосообразно приводить данные к 3НФ для удаления транзитивности признаков.

In [14]:
# сохранение данных в .csv

goods.to_csv('customer_and_transaction/goods.csv')